In [133]:
def calculate(label, df):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    kosong = 0
    
    for index in range(0, len(df)):
        labels = df['label'][index].split('|')
        
        if type(df['pred_multiple'][index]) != float:
            preds = df['pred_multiple'][index].split('|')

            if label in labels and label in preds:
                tp += 1
            elif label not in labels and label not in preds:
                tn += 1
            elif label not in labels and label in preds:
                fp += 1
            elif label in labels and label not in preds:
                fn += 1
        elif label in labels:
            fn += 1
        else:
            tn += 1
    precision = round(tp / (tp + fp), 2)
    recall = round(tp / (tp + fn), 2)
    f1 = round(2 * ((precision * recall) / (precision + recall)), 2)
    accuracy = round((tp+tn)/(tp+tn+fp+fn), 2)
    print(tp, '\t', tn, '\t', fp, '\t', fn, '\t', '\t', precision, '\t', recall, '\t', f1, '\t', '\t', accuracy)
    return tp, tn, fp, fn, precision, recall, f1, accuracy

In [158]:
import pandas as pd
import numpy as np

def run(tipe):
    df = pd.read_csv('Results/Categorization/'+ tipe +'.csv')
    sf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'precision', 'recall', 'f1', 'accuracy'])
    print('tp', '\t', 'tn', '\t', 'fp', '\t', 'fn', '\t', '\t', 'prec', '\t', 'rec', '\t', 'f1', '\t', '\t', 'accuracy')
    precision = []
    recall = []
    f1 = []
    cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'VALUE']
    for c in cat:
        a, b, c, d, e, f, g, h = calculate(c, df)
        precision.append(e)
        recall.append(f)
        f1.append(g)
        sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g, 'accuracy': h}, ignore_index=True)
#     sf.to_excel("Results/Calculation/"+ tipe +".xlsx")
    print('tp', '\t', 'tn', '\t', 'fp', '\t', 'fn', '\t', '\t', round(np.mean(precision),2), '\t', round(np.mean(recall),2), '\t', round(np.mean(f1),2), '\t', '\t')


In [11]:
run("20")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
102 	 475 	 18 	 104 	 	 0.85 	 0.5 	 0.63 	 	 0.83
265 	 259 	 28 	 147 	 	 0.9 	 0.64 	 0.75 	 	 0.75
106 	 521 	 7 	 65 	 	 0.94 	 0.62 	 0.75 	 	 0.9
122 	 502 	 55 	 20 	 	 0.69 	 0.86 	 0.77 	 	 0.89


In [24]:
run("20-noun")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
167 	 398 	 95 	 39 	 	 0.64 	 0.81 	 0.72 	 	 0.81
396 	 213 	 74 	 16 	 	 0.84 	 0.96 	 0.9 	 	 0.87
145 	 483 	 45 	 26 	 	 0.76 	 0.85 	 0.8 	 	 0.9
128 	 416 	 141 	 14 	 	 0.48 	 0.9 	 0.63 	 	 0.78


In [156]:
# run("20-noun-filter")

In [143]:
run("20-beda")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
74 	 487 	 6 	 132 	 	 0.93 	 0.36 	 0.52 	 	 0.8
247 	 263 	 24 	 165 	 	 0.91 	 0.6 	 0.72 	 	 0.73
75 	 526 	 2 	 96 	 	 0.97 	 0.44 	 0.61 	 	 0.86
129 	 402 	 155 	 13 	 	 0.45 	 0.91 	 0.6 	 	 0.76
tp 	 tn 	 fp 	 fn 	 	 0.82 	 0.58 	 0.61 	 	


In [144]:
run("20-noun-beda")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
166 	 399 	 94 	 40 	 	 0.64 	 0.81 	 0.72 	 	 0.81
397 	 213 	 74 	 15 	 	 0.84 	 0.96 	 0.9 	 	 0.87
144 	 479 	 49 	 27 	 	 0.75 	 0.84 	 0.79 	 	 0.89
128 	 419 	 138 	 14 	 	 0.48 	 0.9 	 0.63 	 	 0.78
tp 	 tn 	 fp 	 fn 	 	 0.68 	 0.88 	 0.76 	 	


In [145]:
run("20-noun-filter-beda")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
164 	 407 	 86 	 42 	 	 0.66 	 0.8 	 0.72 	 	 0.82
396 	 218 	 69 	 16 	 	 0.85 	 0.96 	 0.9 	 	 0.88
144 	 488 	 40 	 27 	 	 0.78 	 0.84 	 0.81 	 	 0.9
126 	 448 	 109 	 16 	 	 0.54 	 0.89 	 0.67 	 	 0.82
tp 	 tn 	 fp 	 fn 	 	 0.71 	 0.87 	 0.78 	 	


In [160]:
run("20-ner")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
161 	 426 	 67 	 45 	 	 0.71 	 0.78 	 0.74 	 	 0.84
393 	 225 	 62 	 19 	 	 0.86 	 0.95 	 0.9 	 	 0.88
139 	 495 	 33 	 32 	 	 0.81 	 0.81 	 0.81 	 	 0.91
124 	 463 	 94 	 18 	 	 0.57 	 0.87 	 0.69 	 	 0.84
tp 	 tn 	 fp 	 fn 	 	 0.74 	 0.85 	 0.78 	 	


In [161]:
run("20-ner-ex")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
161 	 427 	 66 	 45 	 	 0.71 	 0.78 	 0.74 	 	 0.84
392 	 226 	 61 	 20 	 	 0.87 	 0.95 	 0.91 	 	 0.88
139 	 495 	 33 	 32 	 	 0.81 	 0.81 	 0.81 	 	 0.91
124 	 475 	 82 	 18 	 	 0.6 	 0.87 	 0.71 	 	 0.86
tp 	 tn 	 fp 	 fn 	 	 0.75 	 0.85 	 0.79 	 	
